# (논문) Proposed method
> decomposition 소개

- toc:true
- branch: master
- badges: true
- comments: true
- author: 김하영
- categories: [Study]

---

## 3. Proposed method

#### 작은 인트로
AI는 이미지를 분류할 때 사람이 통상적으로 사용하는 분류 기준을 이용하지 않기도 한다. 이 때 추가적인 결과를 제공한다면 이미지를 분류한 충분한 증거가 확보되어 AI의 결과를 보완할 수 있을 것이다.     
우리는 CAM을 통해 이미지가 특정 클래스로 분류되는데 중요한 역할을 한 픽셀을 확인하고 이미지에서 분리한다. 그 후에 해당 픽셀을 마스킹(masking)한 이미지를 생성해 그 다음 중요도가 높은 픽셀을 CAM을 통해 확인하는 과정을 올바른 클래스로 분류해낼 때 까지 반복한다.

---

### 3.1 데이터 설명

본 논문에서는 `모드 분해`를 설명하기 위한 예제로 Cat/Dog(https://www.robots.ox.ac.uk/~vgg/data/pets/)을 사용한다. 37종의 고양이, 개로 이루어진 $7,349$개 이미지 데이터로 사이즈는 $512 \times 512$로 통일한다.  
따라서 입력 이미지(아래 사진)인 $\bf X$는 $512 \times 512$ 매트릭스이며, 클래스의 집합 $y$로 분류된다. 
\begin{align}
{\bf{X}} : {\bf{\Lambda}} \to {\mathbb{R}^3} {of size} {512 \times 512}
\end{align}
\begin{align}
y=\{cat, dog\}
\end{align}

<img src="original.png" width=250 height=250 alt="original">

학습은 `resnet34`를 사용하고, 총 epoch 수는 5이다.
예제를 통해 우리는 사람이 고양이, 개를 분류할 때 직관적으로 사용되는 특징에 CAM이 반응하여 모드 분해로 이어질 수 있는지를 확인하고, 이런 특징들이 중요도에 따라 순차적으로 나타나는지를 보고자 한다.

---

### 3.2 CAM(class activation mapping)

CAM은 global average pooling을 사용하여 CNN에서 이미지를 분류하는데 중요한 역할을 한 픽셀을 시각화하는 class activation map을 생성하는 기법이다. <cite>(Zhou et al, 2016)</cite>  

GAP는 마지막 convolutional layer에서 k개의 activation map을 출력한다. 이를 feature map : $f_k(x,y)$라고 한다.  
$w^c_k$는 클래스 $c$에 k번째 feature map의 특징변수가 얼마나 중요한지를 보여주는 가중치이다.  
클래스 $c$에 대한 CAM $M_c$는 k번째 Feature Map의 각 픽셀값을 가중합한 결과이다. <cite>(Zhou et al, 2016)</cite>  

${\bf{M}}_c(x,y) = \sum_k w^c_k f_k(x,y)$

> k : Feture Map의 index  
x,y : Feature Map의 가로(x), 세로(y) 좌표  

예제 이미지를 통해 생성된 CAM은 $16 \times 16$ 매트릭스 형태이며, 각 activation map을 $\bf{M}$$_{0}$, $\bf{M}$$_{1}$으로 지정한다.  
사용되는 이미지가 고양이이기 때문에 모드 분해에는 ${\bf{M}}_{0}$을 사용한다.

${\bf{M}}_{0} = {\bf{M}}_{0_{x,y}}=
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,y} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,y} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{x,1} & a_{x,2} & \cdots & a_{x,y}
 \end{pmatrix}   
  , x \in \{1,2,\dots,16\}  
  , y \in \{1,2,\dots,16\}$

<br/><br/>

${\bf{M}}_{1} = {\bf{M}}_{1_{k,l}}=
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,l} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,l} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{k,1} & a_{k,2} & \cdots & a_{k,l}
 \end{pmatrix}   
  , k \in \{1,2,\dots,16\}  
  , l \in \{1,2,\dots,16\}$

---

### 3.3 가중치(mask) 생성

우리는 이미지에 `가중치(mask)`를 곱하여 `중요도가 높은 픽셀이 마스킹(masking)된 이미지`를 생성한다.  
가중치를 수식으로 표현하면 아래와 같으며, CAM과 동일한 $16 \times 16$ 매트릭스 형태이다.  

$w(x,y) = exp \left(-\frac{1}{2\theta^2} \times {\bf{M}}_{0}(x,y) \right)$

여기서 사용된 $\theta$는 CAM 픽셀들의 분산을 표준화시켜주기 위한 hyper parameter이다.  
$\theta$는 모드 분해가 진행됨에 따라 픽셀들의 분산들이 작아져 약해진 신호<cite>(Yang et al, 2004)</cite>를 보정하기 위해 값을 증가시킨다. 

---

### 3.4 모드 분해 (mode decomposition)

가중치 $w$와 $\bf{X}$ 간 아다마르곱을 통해 모드 분해를 진행한다.  
이 때, $w$는 $\bf{X}$와 형태가 다르기 때문에 bilinear interpolation을 진행하여 $512 \times 512$형태로 재조정한다.  

아다마르곱을 통해 생성된 ${\bf{X}}^{(1)}_{res}$는 `중요도가 높은 픽셀이 마스킹(masking)된 이미지`, `모드 1의 residual`이 되며, 숫자 $1$은 모드 분해 횟수이다.

${\bf{X}}^{(1)}_{res} = {\bf{X}} \odot w$

`모드 1`, `중요도가 높은 픽셀을 추출한 이미지`는 $1-w$와 $\bf{X}$ 간 아다마르곱으로 생성한다.

${\bf{X}}^{(1)} = {\bf{X}} \odot (1-w)$

> 참고 : ${\bf {X}}\approx {\bf {X}}^{(1)}+{\bf X}^{(2)}+{\bf X}^{(3)} \dots +{\bf X}^{(m)}$를 만족한다.  
왜냐하면, ${\bf X}={\bf X}^{(1)}+{\bf X}^{(1)}_{res}$, ${\bf X}^{(1)}={\bf X}^{(2)}+{\bf X}^{(2)}_{res} \dots$를 만족하기 때문이다.  
이 때 $m$은 모드 생성 횟수이다.

3.1~4의 과정을 $\bf{X}$를 올바른 클래스로 분류해내지 못할 때, 즉 accuracy $\leq 0.5$ 일 때까지 진행한다 <cite>(Samek et al, 2016)</cite>.  
따라서 모드 분해는 2개 이상의 결과를 도출하여 이미지 분류에 보다 다양한 증거를 제시한다.

#### 3.4.1 모드 분해 결과

예제를 사용한 모드 분해 분석 결과, 3번의 모드 분해를 진행하였다.  
먼저 1~3회 CAM 결과를 보면, 1차에서는 수염이 있는 고양이 얼굴 아래 부분, 2차에서는 눈이 있는 얼굴 윗 부분, 3차에서는 얼굴을 제외한 귀 부분의 픽셀이 활성화된 것을 확인할 수 있다.  
<img src="fig6.CAM_cat.png" width=500 height=500 alt="CAM_cat">

이를 바탕으로 모드 분해를 진행한 결과는 아래와 같다. 각 시차별 활성화된 픽셀이 `모드`로 분리되고, 분리된 모드가 마스킹된 `모드 residual`이 생성되어 다시 모드 분해를 진행해나가는 과정이다.  
<img src="fig8.Mode_cat.png" width=500 height=500 alt="CAM_cat">

예제에서 우리가 기대한 결과가 도출되었기 때문에 실제 의료 데이터를 이용하여 분석을 진행해보고자 한다.

---

### 4. 실제 데이터 분석

사용된 데이터는 폐렴/정상 2가지로 분류된 총 $5863$개의 chest X-ray 이미지 데이터이다(https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia). 사이즈는 $224 \times 224$ 로 통일한다.  
분석에 사용된 샘플 이미지는 정상으로 분류된 X-ray이미지이다.

앞서 설명한 과정을 통해 분석을 진행하였다. 고양이, 개에 비해 특징이 적어 2차 분해 후 특징이 나타나는 폐 부분이 검게 마스킹되어 분해를 중지하였다.
모드 분해에 따른 accuracy는 아래와 같다.

|No|Accuracy|
|---|---|
|0|0.9999|
|1|0.9906|
|2|0.00|

CAM 결과를 보면, 1차에서는 X-ray의 중앙 부분, 폐와 척추 부분이 활성화되었고, 2차에서는 오른쪽 폐의 아래 부분이 활성화되었다.  
<img src="fig12.CAM_xray.png" width=500 height=500 alt="CAM_cat">

이를 바탕으로 모드 분해를 진행한 결과는 아래와 같다.   
<img src="fig10.mode_xray.png" width=500 height=500 alt="CAM_cat">

CAM을 진행함에 따라 X-ray에서 중요도가 높은 순으로 픽셀이 활성화되었고, 성공적으로 분해가 진행되었다.

---

### 5. Discussions (결과 정리 및 시사점)

우리는 Cat/Dog 데이터에서 모드 분해 아이디어 실현 가능성을 확인하고, 실제 의료 데이터인 Chest X-ray 분석을 통해 이미지를 중요도에 따라 분해해나갈 수 있었다.    

이렇게 한 개의 이미지를 중요도에 따라 여러 특징으로 분해해내는 기법은 여러 분야에서 사용이 확대될 수 있을 것으로 기대된다. 예를 들면, 건물 보수 필요성을 검토하기 위한 노후 건물 이미지 분석을 한다고 하자. 이 때 사람도 AI도 쉽게 발견할 수 있는 커다란 균열이 1차 결과로 제시되었다면, 모드 분해를 통해 크기는 작지만 여전히 건물 보수의 필요성을 보여줄 수 있는 작은 균열들을 추가 증거로 제시하여 보다 풍부한 자료를 통해 적절한 시기에 보수가 이루어질 수 있도록 의사결정을 내릴 수 있을 것이다.  

하지만 여전히 한계점은 존재한다. 본 논문에서 실제 의료 데이터를 사용하여 분석을 진행하였지만, 의료 분야 전문 지식이 부재하여 다양한 의료 데이터를 적용하고 분석 내용을 해석하는데 있어 한계점이 있다. 향후 연구에서는 의료 분야 전문가와 협업하여 실제 의료 이미지 데이터에 모드 분해를 접목할 수 있는지를 살펴보고, 다양한 의료 데이터를 분석하여 더욱 다양한 시사점을 제공할 수 있을 것으로 기대된다.  

다른 한계점으로는 hyper parameter 세팅이 있다. 현재 제안하는 방법론에서 hyper parameter $\theta$는 이미지에 따라 여러 차례 시뮬레이션을 통해 가장 적절한 값을 탐색해야 하는 불편함이 존재한다. 향후 연구에서는 해당 parameter를 더 빠르고 쉽게 찾아낼 수 있는 알고리즘을 구상해볼 수 있을 것이다.

---

### Reference 

\bibitem{Song2019} 송경두, 김명찬, 도신호. 딥러닝 알고리즘 개발과정을 통해 본 영상의학분야에서 딥러닝의 최신 경향. J Korean Soc Radiol 2019; 80(2): 202-12.

\bibitem{Hwang2021} Hwang, Eui Jin, et al. "Deep learning computer-aided detection system for pneumonia in febrile neutropenia patients: a diagnostic cohort study." BMC pulmonary medicine 21.1 (2021): 1-11.

\bibitem{Topol2019} Topol, Eric J. "High-performance medicine: the convergence of human and artificial intelligence." Nature medicine 25.1 (2019): 44-56.

\bibitem{Ko2021} 고학수 외. 인공지능원론: 설명가능성을 중심으로. 박영사, 2021.

\bibitem{Gun2019} Gunning, David, et al. "XAI—Explainable artificial intelligence." Science Robotics 4.37 (2019): eaay7120.

\bibitem{Pet2018} Petsiuk, Vitali, Abir Das, and Kate Saenko. "Rise: Randomized input sampling for explanation of black-box models." arXiv preprint arXiv:1806.07421 (2018).

\bibitem{Zhou2016} Zhou, Bolei, et al. "Learning deep features for discriminative localization." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

\bibitem{Samek2016} Samek, Wojciech, et al. "Evaluating the visualization of what a deep neural network has learned." IEEE transactions on neural networks and learning systems 28.11 (2016): 2660-2673.

\bibitem{Yang2004} Yang, Jian, et al. "Two-dimensional PCA: a new approach to appearance-based face representation and recognition." IEEE transactions on pattern analysis and machine intelligence 26.1 (2004): 131-137.

\bibitem{Rodriguez2010} Rodriguez-Aragon, Licesio J., and Anatoly Zhigljavsky. "Singular spectrum analysis for image processing." Statistics and Its Interface 3.3 (2010): 419-426.
